# Email assistant with Strands Agents SDK

This notebook demonstrates how to build an email assistant with Strands Agents SDK.

We are going to interact with the agents, using a Streamlit UI. The user will be abled to choose between three agents:

* **Image agent**: AI Image Generator and File Manager. It can generate an image based on the user description.
* **Audio RAG**: Audio Knowledge Base. Ask questions about audio content in the Bedrock knowledge base. This KB was created using Bedrock Data Automation (BDA) as parser and Amazon earning calls audio file as source data.
* **Email assistant agent**: Enhanced Email Assistant with RAG and Image Generation. This assistant can create professional emails using tools and other agents - Search web resources, retrieve relevant audio context (using Audio RAG agent) and generate appropriate images (using Image agent).
* **Report Writing**: report_agent process with planning and writing


By combining these components, our multi-agent system will be able to provide accurate and informative responses to a diverse set of queries.

In this nottebbok, you will find the following fils:

* **app.py**: Streamlit UI application
* **generate_img_streamlit.py**: Strands tool to generate an image using Nova Canvas model and save it to a file.
* **ui_utils.py**: Remaining Strands tools, agents and helper functions 

### Pre-requisite: 

### Please note, you have to create Bedrock knowledge base for audio files for this notebook. Please run notebooks 1-2 in folder <mark> audio-video-rag. 

## 1. Install Required Packages

First, let's install the necessary packages for our multi-agent system.

In [ ]:
!pip install -r requirements.txt -q --no-cache-dir

## 2. Setting Up Environment

We need to import required packages and set up the necessary environment variables.

In [ ]:
import os
import sys
import json
import boto3
import sagemaker
from generate_img_streamlit import generate_img_streamlit

from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import retrieve, http_request, think, editor 

sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()["Account"]
session = sagemaker.Session()

region_name = boto3.session.Session().region_name

os.environ["KNOWLEDGE_BASE_ID"] = "HU4YII3DEY"
os.environ["MIN_SCORE"] = "0.4"
os.environ["AWS_REGION"] = region_name


## 3. Streamlit UI

Running these code cells will start the Streamlit UI application and show the application endpoint for access

In [ ]:

def get_current_notebook_url():
    try:
        # Create SageMaker client
        sagemaker = boto3.client('sagemaker')
        
        # Get the current notebook instance name from environment
        notebook_name = os.environ.get('NOTEBOOK_INSTANCE_NAME')
        
        if not notebook_name:
            # Alternative way to get instance name from resource metadata
            try:
                with open('/opt/ml/metadata/resource-metadata.json') as f:
                    metadata = json.load(f)
                    notebook_name = metadata.get('ResourceName')
            except:
                pass
        
        if notebook_name:
            # Get presigned URL for the notebook instance
            response = sagemaker.create_presigned_notebook_instance_url(
                NotebookInstanceName=notebook_name
            )
            
            url = response['AuthorizedUrl']
            return url
        else:
            print("Could not determine notebook instance name")
            
    except Exception as e:
        print(f"Error getting notebook URL: {str(e)}")
    
    return None

def get_streamlit_url(text):
    question_mark_index = text.find('?')
    if question_mark_index != -1:
        return text[:question_mark_index] + "/proxy/absolute/8501"
    return text

# Get and print the current notebook URL
URL = get_current_notebook_url()

streamlit_URL=get_streamlit_url(URL)
print(streamlit_URL)


In [ ]:
# Kill any existing Streamlit processes
!pkill -f streamlit

# Start Streamlit
!nohup streamlit run ./app.py --server.baseUrlPath="/proxy/absolute/8501"

### 4. Test

now we can test the email assistant application:

1. Choose 'Audio RAG' agent and run the query:
   "can you find an audio file about amazon earning calls?"
   
3. Choose 'Image agent' and run the query:
   "Can you shown me an Amazon delivery truck?"

4. Choose 'Email assistant agent' and run the queries in order:
    1. "Help me to write an email about Amazon Q3 earning call"
    2. "Use my name in the email, John smith"
    3. "generate an image of Amazon logo and add it at the end of the email"

